### Limpeza de dados
O objetivo desse notebook é realizar uma limpeza de dados na tabela de informações dos pacientes que realizaram algum exame CK (`pacients_with_ck.csv`), na tabela dos exames CK realizados (`ck_events.csv`) e na tabela dos exames de glicose realizados (`glucose_events.csv`).

In [2]:
import pandas as pd

#### Limpeza da tabela pacients_with_ck.csv
Os passos para limpeza dessa tabela são:
1. Filtragem dos dados para incluir somente os resultados relativos ao BMI (IMC) em kg/m^2 (dados de BMI sem unidade indicada foram desconsiderados)
2. Verificação dos valores de BMI extremamente elevados e calculando quanto da população total eles representam
3. Exportação tabela filtrada considerando apenas resultados de BMI < 100

Ao final do processamento uma nova tabela `bmi_norm.csv` é criada

In [ ]:
df = pd.read_csv('../data/interim/pacients_with_ck.csv')

bmi = df[df['result_name'] == 'BMI (kg/m2)']
bmi['result_value'] = bmi['result_value'].astype(float)

bmi_norm = bmi[bmi['result_value'] < 100]
bmi_abs = bmi[bmi['result_value'] >= 100]
abs_percent = 100*(len(bmi_abs)/len(bmi))
print(f"Porcentagem de medições com BMI acima de 100 kg/m2: {abs_percent:.2f}%")

bmi_norm.to_csv('../data/processed/bmi_norm.csv', index=False)

/media/salathiel/HD/mconda/envs/rl/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Porcentagem de medições com BMI acima de 100 kg/m2: 0.03%


#### Limpeza da tabela ck_events.csv
Os passos para limpeza dessa tabela são:
1. Remoção de outliers, considerados CK > 10000 IU/L
2. Cálculo de quanto da população total os outliers representam
3. Exportação da tabela filtrada considerando apenas resultados de CK < 10000 IU/L

Ao final do processamento uma nova tabela `ck_norm.csv` é criada

In [ ]:
ck = pd.read_csv('../data/interim/ck_events.csv')

ck_norm = ck[ck['valuenum'] <= 10000]
ck_abs = ck[ck['valuenum'] > 10000]
abs_percent = 100*(len(ck_abs)/len(ck))
print(f"Porcentagem de medições com CK acima de 10000 IU/L: {abs_percent:.2f}%")

# Exportando somente os resultados de CK <= 10 000
ck_norm.to_csv('../data/processed/ck_norm.csv', index=False)

Porcentagem de medições com CK acima de 10000 IU/L: 4.22%


#### Limpeza da tabela glucose_events.csv
Os passos para limpeza dessa tabela são:
1. Remoção de dados com unidade diferente de mg/dL ou sem unidade
2. Remoção de valores nulos
3. Remoção de valores iguais a 0
4. Remoção de valores acima do limite de acurácia (1000 mg/dL em geral, 500 mg/dL para medições com fingerstick)
5. Exportação da tabela filtrada

Ao final do processamento uma nova tabela `glucose_norm.csv` é criada

In [4]:
glucose = pd.read_csv('../data/interim/glucose_events.csv')
tam = len(glucose)

# Removendo unidades incompatíveis
glucose_sem_u = glucose[glucose['valueuom'] == 'mg/dL']
tam_sem_u = len(glucose_sem_u)
tam_u = tam - tam_sem_u
u_percent = 100*(tam_u / tam)
print(f"Porcentagem de linhas removidas após o passo 1: {u_percent:.2f}%")

# Removendo valores nulos
glucose_sem_un = glucose_sem_u[~glucose_sem_u['valuenum'].isnull()]
tam_sem_un = len(glucose_sem_un)
tam_un = tam_sem_u - tam_sem_un
n_percent = 100*(tam_un / tam_sem_u)
print(f"Porcentagem de linhas removidas após o passo 2: {n_percent:.2f}%")

# Removendo valores iguais a 0
glucose_sem_unz = glucose_sem_un[glucose_sem_un['valuenum'] != 0]
tam_sem_unz = len(glucose_sem_unz)
tam_z = tam_sem_un - tam_sem_unz
z_percent = 100*(tam_z / tam_sem_un)
print(f"Porcentagem de linhas removidas após o passo 3: {z_percent:.2f}%")

# Removendo valores acima do limite
glucose_sem_unzl = glucose_sem_unz[((glucose_sem_unz['itemid'] == 225664) & (glucose_sem_unz['valuenum'] < 500)) |
                                   ((glucose_sem_unz['itemid'] != 225664) & (glucose_sem_unz['valuenum'] < 1000))]
tam_sem_unzl = len(glucose_sem_unzl)
tam_l = tam_sem_unz - tam_sem_unzl
l_percent = 100*(tam_l / tam_sem_unz)
print(f"Porcentagem de linhas removidas após o passo 4: {l_percent:.2f}%")

glucose_sem_unzl.to_csv('../data/processed/glucose_norm.csv', index=False)

Porcentagem de linhas removidas após o passo 1: 58.94%
Porcentagem de linhas removidas após o passo 2: 0.00%
Porcentagem de linhas removidas após o passo 3: 0.00%
Porcentagem de linhas removidas após o passo 4: 0.04%
